# Implementation of the Polyphase filterbank

In [18]:
import numpy as np

class analysisBuffer:
    def __init__(self,bufferVal=np.zeros(512)):
        assert (len(bufferVal)==512),"Input not length 512!"
        self.bufferVal = bufferVal
        
    def pushBlock(self,sampleBlock):
        assert (len(sampleBlock)==32),"Sample block length not 32!"
        self.bufferVal[32:] = self.bufferVal[:-32]
        self.bufferVal[0:32] = sampleBlock

def polyphaseFilterbank(x):
    
    C = np.load('data/mpeg_analysis_window.npy') # analysis window defined by MPEG
    
    M = calcAnalysisMatrixCoeff()
    
    subbandSamples = np.zeros(32)
    for n in range(32):
        for k in range(63):
            for m in range(7):
                subbandSamples[n] = subbandSamples[n] + (M[n,k] * (C[k+64*m]*x.bufferVal[k+64*m]))
        
    return subbandSamples
    
def calcAnalysisMatrixCoeff():
    M = np.zeros([32,64])
    for n in range(32):
        for k in range(64):
            M[n,k] = np.cos((2*n+1)*(k-16)*np.pi/64)
            
    return M
     

Simple test

In [19]:
# test pushBlock in Buffer and Filterbank just with random values
myBuffer = analysisBuffer(np.random.rand(512))
myBuffer.pushBlock(np.random.rand(32))

myst = polyphaseFilterbank(myBuffer)
print(myst)

[ 0.43198655  0.10213128 -0.01506771  0.00115202  0.02467189 -0.03197714
  0.08208882 -0.01238919 -0.04730537 -0.01520704  0.0573132   0.02986398
 -0.00757987 -0.0156165  -0.04741508 -0.02848638 -0.01739938 -0.04036587
 -0.00354687 -0.02200082  0.02172196 -0.03611722 -0.05960432 -0.05883446
  0.10991358  0.05770042 -0.10469166 -0.00429397 -0.00424095 -0.00581947
 -0.00534673  0.11426906]


In [20]:
# test with a sine tone at 10kHz, assuming roughly 48kHz sampling rate
# --> maximum value in filterbank output seen in 13th entry

myBuffer = analysisBuffer(np.sin(2*np.pi*10000*np.linspace(0,0.01,512)))
myst = polyphaseFilterbank(myBuffer)
print(myst)

[ 1.21202680e-02 -1.44946970e-02 -9.28328898e-03  1.63199509e-02
  6.09067813e-03 -1.75211143e-02 -2.66505598e-03  1.80719505e-02
 -8.64472328e-04 -1.79703665e-02  4.35845671e-03  1.74879278e-02
  9.78679207e-01 -1.50207993e-02  1.05408444e-02  1.31735898e-02
 -1.32306672e-02 -1.06348660e-02  1.53482354e-02  7.64692128e-03
 -1.68664303e-02 -4.34626024e-03  1.77265416e-02  8.73449724e-04
 -1.79064471e-02  2.63321283e-03  1.73947968e-02 -6.04335340e-03
 -1.62124859e-02  9.22353398e-03  1.44085065e-02 -1.20454323e-02]


# Implementation of Calculation of scale factors